In [4]:
import nltk
import speech_recognition as sr

def Speech_to_text(audio_file):

    r = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = r.record(source)
    text = r.recognize_google(audio)
    print(text)
    return text

text = Speech_to_text("Full_Call.wav")

hello thank you for calling Amazon customer support my name is Alex how can I assist you today I need to return a product I purchase last week sure I would be happy to help with that could you please provide your order number yes the order number is 56789 thank you just to confirm is this for the wireless headphones you purchase on January 15th yes that's correct I will process the return for you before we proceed I want to inform you of Return policy returns must be initiated within 1330 days of purchase and the item must be in its original packaging show your security please do not share any sensitive information like your credit card details account password during this school my credit card number with your call you yesterday 1234567 8987 6543200 I didn't know that I have initiated the return process you will receive a confirmation email within 24 hours and return label with will be sent to you registered email is there anything else I can help you with today no that's all thank yo

In [2]:
# Lists of phrases
greetings = [
    "hello", "hi", "good morning", "good afternoon", "good evening", 
    "greetings", "hey", "how can I help you?", "how may I assist you today?", 
    "welcome to", "thank you for calling", "this is", "thank you for reaching out.", 
    "welcome to our customer support"
]

closing_statements = [
    "thank you for calling", "have a great day", "goodbye", "take care", 
    "have a nice day", "stay safe", "thank you for choosing", 
    "if you have further questions, feel free to reach out", 
    "we’re here if you need any more help", "it was my pleasure assisting you", 
    "let us know if there’s anything else we can do for you", 
    "thank you" ,"goodbye"
]

disclaimers = [ 
    'please do not share any sensitive information',
    "for your security", "avoid disclosing personal information" "avoid disclosing personal information during this call", 
    "your privacy is important to us. do not share confidential details", 
    "returns must be initiated within", "by proceeding with this call, you agree to our terms and conditions", 
    "all calls may be monitored or recorded for quality and training purposes", 
    "we value your privacy and ensure all data is handled securely", 
    "please review our privacy policy on"
]

# Function to detect phrases in text
def detect_phrases(text, phrase_list):
    detected_phrases = []
    for phrase in phrase_list:
        if phrase.lower() in text.lower():
            detected_phrases.append(phrase)
    return detected_phrases


# Detecting phrases
detected_greetings = detect_phrases(text, greetings)
detected_closing_statements = detect_phrases(text, closing_statements)
detected_disclaimers = detect_phrases(text, disclaimers)

# Output results
print("Detected Greetings:", detected_greetings)
print("Detected Closing Statements:", detected_closing_statements)
print("Detected Disclaimers:", detected_disclaimers)


Detected Greetings: ['hello', 'hi', 'thank you for calling']
Detected Closing Statements: ['thank you for calling', 'have a great day', 'thank you for choosing', 'thank you']
Detected Disclaimers: ['please do not share any sensitive information', 'returns must be initiated within']


In [3]:
from pyannote.audio import Pipeline
import speech_recognition as sr

# Initialize the pyannote pipeline for speaker diarization
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token="<HUGGING_FACE_TOKEN>")

# Input audio file
audio_file = "Full_Call.wav"

# Perform speaker diarization
diarization = pipeline(audio_file)

# Prepare the speech recognizer
recognizer = sr.Recognizer()

# Map diarized segments to speakers
speaker_segments = []

with sr.AudioFile(audio_file) as source:
    audio = recognizer.record(source)
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        # Extract audio segment for each speaker
        segment_audio = audio.get_segment(turn.start, turn.end)
        # Transcribe the segment
        try:
            text = recognizer.recognize_google(segment_audio)
        except sr.UnknownValueError:
            text = "[Unintelligible]"
        except sr.RequestError as e:
            text = f"[Error: {e}]"

        # Store speaker and their text
        speaker_segments.append({"speaker": speaker, "start": turn.start, "end": turn.end, "text": text})

# Display the results
for segment in speaker_segments:
    print(f"{segment['speaker']} [{segment['start']:.2f}s - {segment['end']:.2f}s]: {segment['text']}")


c:\Semester 6\MLops\Customer Service Agent\Customer-Service-Assistant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Semester 6\MLops\Customer Service Agent\Customer-Service-Assistant\.venv\Lib\site-packages\pyannote\database\util.py:182: SyntaxWarning: invalid escape sequence '\s'
  sep="\s+",
c:\Semester 6\MLops\Customer Service Agent\Customer-Service-Assistant\.venv\Lib\site-packages\pyannote\database\util.py:216: SyntaxWarning: invalid escape sequence '\s'
  sep="\s+",
c:\Semester 6\MLops\Customer Service Agent\Customer-Service-Assistant\.venv\Lib\site-packages\pyannote\database\util.py:253: SyntaxWarning: invalid escape sequence '\s'
  sep="\s+",
c:\Semester 6\MLops\Customer Service Agent\Customer-Service-Assistant\.venv\Lib\site-packages\pyannote\database\util.py:284: SyntaxWarning: invalid escape 


Could not download 'pyannote/speaker-diarization-3.1' pipeline.
It might be because the pipeline is private or gated so make
sure to authenticate. Visit https://hf.co/settings/tokens to
create your access token and retry with:

   >>> Pipeline.from_pretrained('pyannote/speaker-diarization-3.1',
   ...                          use_auth_token=YOUR_AUTH_TOKEN)

If this still does not work, it might be because the pipeline is gated:
visit https://hf.co/pyannote/speaker-diarization-3.1 to accept the user conditions.


TypeError: 'NoneType' object is not callable

<!-- PII Detection -->

PII Detection

In [5]:
import re



def detect_pii(text):
    """
    Detects critical PII in the given text based on regex patterns.
    Args:
        text (str): Input text to scan for PII.
    Returns:
        dict: Dictionary of detected PII types and matches.
    """

    # Define regex patterns for critical PII
    pii_patterns = {
        # Financial Information
        "credit_card": r"\b(?:\d{4}[ -]?){3}\d{4}\b",  # Credit card numbers
        "atm_pin": r"\b\d{4}\b",  # ATM PINs (4 digits)

        # Contact Information
        "email": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b",  # Email addresses
        "phone_number": r"\+?\d{1,3}[ -]?\(?\d{1,4}\)?[ -]?\d{1,4}[ -]?\d{1,4}\b",  # Phone numbers

        # Identity Information
        "ssn": r"\b\d{3}-\d{2}-\d{4}\b",  # Social Security Numbers (SSNs)

        # Network Information
        "ip_address": r"\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b",  # IPv4 addresses

        # Account Numbers
        "bank_account_number": r"\b\d{9,18}\b",  # Bank account numbers (9 to 18 digits)
    }

    detected_pii = {}
    for pii_type, pattern in pii_patterns.items():
        matches = re.findall(pattern, text)
        if matches:
            detected_pii[pii_type] = matches
    return detected_pii

# Example Usage
sample_text = """
My email is john.doe@example.com and my phone number is +1-800-555-1234. 
My credit card number is 4111 1111 1111 1111, and my bank account number is 123456789012.
My SSN is 123-45-6789, and I once logged in from IP 192.168.1.1.
"""


detected = detect_pii(sample_text)
print("Detected PII:", detected)

detected = detect_pii(text)
print("Detected PII:", detected)


Detected PII: {'credit_card': ['4111 1111 1111 1111'], 'atm_pin': ['1234', '4111', '1111', '1111', '1111', '6789'], 'email': ['john.doe@example.com'], 'phone_number': ['+1-800-555-1234', '4111 1111 1111', '1111', '123456789012', '123-45-6789'], 'ssn': ['123-45-6789'], 'ip_address': ['192.168.1.1'], 'bank_account_number': ['123456789012']}
Detected PII: {'atm_pin': ['1330', '8987'], 'phone_number': ['56789', '1330', '1234567 8987', '6543200']}
